In [2]:
import pyowm
import pandas as pd
import numpy as np

from auth import (
    OWM_api_key)

In [3]:
owm = pyowm.OWM(OWM_api_key)

In [4]:
park_loc = pd.DataFrame({
    'name': ['Prospect Park', 'Wooster Square'],
    'lat': [40.662551, 41.304745],
    'long': [-73.969256, -72.917757]})

In [5]:
park_name = 'Wooster Square'
park_lat = park_loc.loc[park_loc.name == park_name, 'lat'].iloc[0]
park_long = park_loc.loc[park_loc.name == park_name, 'long'].iloc[0]

park_fcstr = owm.three_hours_forecast_at_coords(
    float(park_lat),float(park_long))

In [6]:
OWM_forcast_cols = [
    'park_name',
    'reception_time',
    'reference_time',
    'sunset_time',
    'sunrise_time',
    'clouds',
    'rain',
    'snow',
    'wind_speed',
    'wind_deg',
    'humidity',
    'press',
    'press_sea',
    'temp',
    'temp_max',
    'temp_min',
    'status',
    'detailed_status',
    'visibility_distance',
    'dewpoint',
    'humidex',
    'heat_index'
]

In [7]:
park_forcast = park_fcstr.get_forecast()
park_forcast

<pyowm.weatherapi25.forecast.Forecast - reception time=2020-04-02 18:09:45+00, interval=3h>

In [9]:
forcast_df = pd.DataFrame(columns=OWM_forcast_cols)

for x in park_forcast.get_weathers():
    x_sunrise = x.get_sunrise_time(timeformat='unix')
    if x_sunrise == 0:
        x_sunrise = np.nan

    x_sunset = x.get_sunset_time(timeformat='unix')
    if x_sunset == 0:
        x_sunset = np.nan
    
    x_rain = 0
    if x.get_rain():
        x_rain = x.get_rain()['3h']
    
    x_snow = 0
    if x.get_snow():
        x_snow = x.get_snow()['3h']
    
    x_vis_dist = x.get_visibility_distance()
    if x_vis_dist is None:
        x_vis_dist = np.nan
        
    x_dew = x.get_dewpoint()
    if x_dew is None:
        x_dew = np.nan
    
    x_humid = x.get_humidex()
    if x_humid is None:
        x_humid = np.nan
        
    x_heat_idx = x.get_heat_index()
    if x_heat_idx is None:
        x_heat_idx = np.nan
    
    to_add_dict = {
        'park_name': [park_name],
        'reception_time': [park_forcast.get_reception_time('unix')],
        'reference_time': [x.get_reference_time()],
        'sunset_time': [x_sunrise],
        'sunrise_time': [x_sunset],
        'clouds': [x.get_clouds()],
        'rain': [x_rain],
        'snow': [x_snow],
        'wind_speed': [x.get_wind(unit='meters_sec')['speed']],
        'wind_deg': [x.get_wind(unit='meters_sec')['deg']],
        'humidity': [x.get_humidity()],
        'press': [x.get_pressure()['press']],
        'press_sea': [x.get_pressure()['sea_level']],
        'temp': [x.get_temperature(unit='fahrenheit')['temp']],
        'temp_max': [x.get_temperature(unit='fahrenheit')['temp_max']],
        'temp_min': [x.get_temperature(unit='fahrenheit')['temp_min']],
        'status': [x.get_status()],
        'detailed_status': [x.get_detailed_status()],
        'visibility_distance': [x_vis_dist],
        'dewpoint': [x_dew],
        'humidex': [x_humid],
        'heat_index': [x_heat_idx]
    }
    to_add = pd.DataFrame(to_add_dict)
    forcast_df = forcast_df.append(to_add, ignore_index=True)

In [10]:
forcast_df.head()

,park_name,reception_time,reference_time,sunset_time,sunrise_time,clouds,rain,snow,wind_speed,wind_deg,...,press_sea,temp,temp_max,temp_min,status,detailed_status,visibility_distance,dewpoint,humidex,heat_index
0,Wooster Square,1585850985,1585861200,NaN,NaN,100,0.42,0,8.12,332,...,1001,47.71,47.71,44.82,Rain,light rain,NaN,NaN,NaN,NaN
1,Wooster Square,1585850985,1585872000,NaN,NaN,100,0.58,0,8.33,329,...,1002,47.97,47.97,45.79,Rain,light rain,NaN,NaN,NaN,NaN
2,Wooster Square,1585850985,1585882800,NaN,NaN,100,0.77,0,8.13,332,...,1001,47.05,47.05,45.61,Rain,light rain,NaN,NaN,NaN,NaN
3,Wooster Square,1585850985,1585893600,NaN,NaN,100,0.48,0,7.42,338,...,1001,46.69,46.69,45.97,Rain,light rain,NaN,NaN,NaN,NaN
4,Wooster Square,1585850985,1585904400,NaN,NaN,100,0.20,0,6.94,341,...,1001,46.22,46.22,46.22,Rain,light rain,NaN,NaN,NaN,NaN
